<a href="https://colab.research.google.com/github/DrSubbiah/1.Bayesian-Inference/blob/master/16_STAN_Generated_Quantities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center><font color="darkblue">  Generated Quantities in STAN </center>


STAN's *generated quantities block* computes QoIs based on the

- data

- transformed data

- parameters

- transformed parameters

In [ ]:
import numpy as np
import pandas as pd
import statistics as stat
import scipy
import pystan

In [ ]:
#For plots
import arviz as az
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Data Sets/BirthWt.csv"
brtwt_da = pd.read_csv(path)

In [ ]:
np.where(pd.isnull(brtwt_da))

In [ ]:
np.where(brtwt_da.applymap(lambda x: x == ''))

In [ ]:
brtwt_da.columns

# <font color="darkorange"> Linear Model I

- Response: Bweight

- Predictor: gestwks

In [ ]:
brtwt_LM_code1 = """
data {
    int<lower=0> n;
    real a_i;
    real<lower=0> b_i;
    real a_p1;
    real<lower=0> b_p1;
    vector[n] y;
    vector[n] x1;
    real<lower=0> g1;
    real<lower=0> g2;
   }

parameters {
    real b0;
    real b1;
    real<lower=0> tau;
}

transformed parameters {
   real<lower=0> sig;
   sig=sqrt(1/tau);
}

model {
      y ~ normal(b0+b1*x1, sig);
      b0 ~ normal(a_i, b_i);
      b1 ~ normal(a_p1, b_p1);
      tau ~ gamma(g1,g2);
}
generated quantities{
   real Rsq;
   real<lower=0> rss;             // residual sum of squares
   real<lower=0> totalss;         // total SS


  //computation of R square
  vector[n] yhat;                // linear predictor
  yhat = b0+b1*x1;
  rss = dot_self(y-yhat);
  totalss = dot_self(y-mean(y));
  Rsq = 1 - (rss/totalss);
}
"""
# posterior
posterior1 = pystan.StanModel(model_code=brtwt_LM_code1)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_abfa346bc905cea1cdd614e7f2ec7cf2 NOW.


In [ ]:
brtwt_LM_data1 = {
             'n': len(brtwt_da),
             'x1': brtwt_da['gestwks']-np.mean(brtwt_da['gestwks']),
             'y': brtwt_da['bweight'],
             'a_i':3000,
             'b_i':10,
             'a_p1':100,
             'b_p1':10,
             'g1':3,
             'g2':1,}
print(brtwt_LM_data1)

In [ ]:
brtwt_LM_model1= posterior1.sampling(data=brtwt_LM_data1,
                  iter=10000,
                  chains=4,
                  seed=1,
                  warmup=3000,
                  thin=1,
                  control={"max_treedepth":15,"adapt_delta" : 0.9999})

In [ ]:
post_draws_A=brtwt_LM_model1.extract()
post_draws_yhat=post_draws_A['yhat']
# del post_draws_A['yhat']
# post_draws_mod1=pd.DataFrame(post_draws_A)

In [ ]:
post_draws_yhat

array([[2867.45795428, 3093.2072176 , 2544.044008  , ..., 3059.58451901,
        3298.14254553, 2933.10134006],
       [2873.5139356 , 3105.16109843, 2541.65051342, ..., 3070.65997705,
        3315.45054337, 2940.87231299],
       [2875.69885659, 3103.95050181, 2548.69993645, ..., 3069.95510285,
        3311.15749365, 2942.06988514],
       ...,
       [2872.26970671, 3104.8260162 , 2539.10381869, ..., 3070.18948812,
        3315.94078507, 2939.89244583],
       [2864.7714867 , 3108.26341645, 2515.93896017, ..., 3071.9981563 ,
        3329.30554848, 2935.5740862 ],
       [2849.88472813, 3093.86221616, 2500.35657886, ..., 3057.52463781,
        3315.34513832, 2920.8285183 ]])

In [ ]:
az.plot_trace(brtwt_LM_model1,var_names=['b0','b1','sig'], compact=False,legend=True,figsize=(20,15))
plt.show()

In [ ]:
func_quan = {
    "1%": lambda x: np.percentile(x, 1),
    "5%": lambda x: np.percentile(x, 5),
    "25%": lambda x: np.percentile(x, 25),
    "75%": lambda x: np.percentile(x, 75),
    "95%": lambda x: np.percentile(x, 95),
    "99%": lambda x: np.percentile(x, 99),
}
summ_mod1=az.summary(brtwt_LM_model1,var_names=['b0','b1','sig',"Rsq"],round_to=3,hdi_prob=0.95,stat_funcs=func_quan)
summ_mod1

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,1%,5%,25%,75%,95%,99%
b0,3030.049,8.886,3012.926,3047.724,0.067,0.047,17743.664,15400.350,1.0,3009.241,3015.505,3024.107,3036.070,3044.751,3050.461
b1,166.383,6.326,154.098,178.901,0.047,0.033,18423.010,16425.038,1.0,151.623,155.967,162.096,170.698,176.696,180.969
sig,459.779,13.256,433.168,485.351,0.098,0.069,18451.078,16099.507,1.0,430.539,438.423,450.732,468.578,482.385,491.921
Rsq,0.500,0.008,0.484,0.514,0.000,0.000,17217.519,15917.791,1.0,0.480,0.486,0.494,0.505,0.512,0.516


## <font color="darkred"> Remarks

- If the prior for the predictor is around zero, then this data set brings negative $R^2$

- This shows poor fit

- Impact of centering the numeric predictor can also be seen from $R^2$



# <font color="darkorange"> Linear Model - II

- Response: Bweight

- Predictors:

  - gestwks - Numeric

  - sex - categorical (binary) - dummy coded


In [ ]:
dumm_sex=pd.get_dummies(brtwt_da['sex'],prefix='sex_', columns=['sex'])
brtwt_da = pd.concat([brtwt_da, dumm_sex], axis=1)

In [ ]:
brtwt_LM_code2 = """
data {
    int<lower=0> n;
    //real a_i;
    //real<lower=0> b_i;
    real a_p1;
    real<lower=0> b_p1;
    real a_p2;
    real<lower=0> b_p2;
    real a_p3;
    real<lower=0> b_p3;
    vector[n] y;
    vector[n] x1;
    vector[n] x2;
    vector[n] x3;
    real<lower=0> g1;
    real<lower=0> g2;
    real b1_L;
    }

parameters {
   //real b0;
    real b1;
    real b2;
    real b3;
   real<lower=0> tau;
}

transformed parameters {
   real<lower=0> sig;
   sig=sqrt(1/tau);
}

model {
      y ~ normal(b1*x1+b2*x2+b3*x3, sig);
      //b0 ~ normal(a_i, b_i);
      b1 ~ normal(a_p1, b_p1);
      b2 ~ normal(a_p2, b_p2);
      b3 ~ normal(a_p3, b_p3);
      tau ~ cauchy(g1,g2); //gamma
}
generated quantities{
   real Rsq;
   real<lower=0> rss;             // residual sum of squares
   real<lower=0> totalss;         // total SS
   int<lower=0, upper=1> b1_gt_L= b1 > b1_L; //Posterior Probability about b1
   real b2_b3= b2 - b3;                      //Posterior Probability about b2 and b3

  //computation of R square
  vector[n] yhat;                // linear predictor
  yhat = b1*x1+b2*x2+b3*x3;
  rss = dot_self(y-yhat);
  totalss = dot_self(y-mean(y));
  Rsq = 1 - (rss/totalss);
}
"""
# posterior
posterior2 = pystan.StanModel(model_code=brtwt_LM_code2)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_65a997f00298c678063df17944377cca NOW.


In [ ]:
brtwt_LM_data2 = {
             'n': len(brtwt_da),
             'x1': brtwt_da['gestwks']-np.mean(brtwt_da.gestwks),
             'x2': brtwt_da['sex__female'],
             'x3': brtwt_da['sex__male'],
             'y': brtwt_da['bweight'],
            #  'a_i':3000,
            #  'b_i':10,
             'a_p1':200,
             'b_p1':100,
             'a_p2':3000,
             'b_p2':100,
             'a_p3':3000,
             'b_p3':100,
             'g1':0,
             'g2':10,
             'b1_L':10}
#print(brtwt_LM_data2)

In [ ]:
brtwt_LM_model2= posterior2.sampling(data=brtwt_LM_data2,
                  iter=10000,
                  chains=4,
                  seed=1,
                  warmup=3000,
                  thin=1,
                  control={"max_treedepth":15,"adapt_delta" : 0.9999})

In [ ]:
az.plot_trace(brtwt_LM_model2,var_names=['b1','b2','b3','sig'], compact=False,legend=True,figsize=(20,15))
plt.show()

In [ ]:
func_quan = {
    "1%": lambda x: np.percentile(x, 1),
    "5%": lambda x: np.percentile(x, 5),
    "25%": lambda x: np.percentile(x, 25),
    "75%": lambda x: np.percentile(x, 75),
    "95%": lambda x: np.percentile(x, 95),
    "99%": lambda x: np.percentile(x, 99),
}
summ_mod1=az.summary(brtwt_LM_model2,var_names=['b1','b2','b3','b2_b3','b1_gt_L','sig',"Rsq"],round_to=3,hdi_prob=0.95,stat_funcs=func_quan)
summ_mod1

/usr/local/lib/python3.7/dist-packages/arviz/stats/diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,1%,5%,25%,75%,95%,99%
b1,206.432,7.309,192.602,221.031,0.048,0.034,22979.481,18420.533,1.0,189.419,194.408,201.489,211.394,218.453,223.402
b2,3044.036,23.904,2996.705,3090.283,0.153,0.108,24401.696,19068.762,1.0,2989.231,3004.691,3028.000,3059.898,3083.625,3099.866
b3,3197.242,23.068,3151.922,3242.363,0.147,0.104,24727.126,18235.056,1.0,3143.598,3159.168,3181.793,3212.791,3235.367,3251.052
b2_b3,-153.206,33.438,-218.916,-87.873,0.213,0.154,24589.107,18410.281,1.0,-231.586,-207.786,-175.828,-130.742,-98.557,-74.598
b1_gt_L,1.000,0.000,1.000,1.000,0.000,0.000,28000.000,28000.000,NaN,1.000,1.000,1.000,1.000,1.000,1.000
sig,433.797,12.061,410.681,457.995,0.079,0.056,23583.385,18340.472,1.0,406.811,414.275,425.566,441.825,454.302,462.575
Rsq,0.557,0.002,0.554,0.559,0.000,0.000,13964.488,16628.277,1.0,0.551,0.554,0.556,0.558,0.559,0.559


# <font color="darkorange"> A note on Posterior Probabilities

Instead of including a variable in the generated quantities for finding probabilties associated with a parameter, it is possibe to use **extract** function from Stan.

Once the posterior draws are saved it is possible to count the number of points satisfying the condition (of interest) and divide by the number of runs

In [ ]:
post_draws_A=brtwt_LM_model1.extract()
del post_draws_A['yhat']
post_draws_mod1=pd.DataFrame(post_draws_A)

In [ ]:
post_draws_mod1

,b0,b1,tau,sig,Rsq,rss,totalss,lp__
0,3019.118701,160.105878,0.000005,465.453951,0.487943,1.396482e+08,2.727201e+08,-4316.714597
1,3029.136953,164.288785,0.000005,450.143952,0.497625,1.370078e+08,2.727201e+08,-4315.874689
2,3029.040730,161.880617,0.000005,448.421500,0.494907,1.377490e+08,2.727201e+08,-4316.292231
3,3033.505663,155.929161,0.000005,460.431509,0.489723,1.391629e+08,2.727201e+08,-4316.982447
4,3033.942412,162.160967,0.000005,458.559083,0.497476,1.370485e+08,2.727201e+08,-4315.830928
...,...,...,...,...,...,...,...,...
27995,3028.109288,169.836705,0.000005,458.023302,0.502734,1.356145e+08,2.727201e+08,-4315.675721
27996,3016.746639,159.961855,0.000005,454.261380,0.486532,1.400331e+08,2.727201e+08,-4317.464622
27997,3028.503499,164.933570,0.000004,476.956197,0.498017,1.369009e+08,2.727201e+08,-4316.365963
27998,3028.351944,172.689330,0.000005,434.363095,0.505420,1.348818e+08,2.727201e+08,-4317.690596


In [ ]:
post_draws_mod1.b1[post_draws_mod1.b1>2650].count()/len(post_draws_mod1)